Using Python, you can interact with the OpenAI Assistants API to upload, manage, and check files. Below are the key principles you need to understand:

# 1. Upload file to OpenAI

To upload a file so that an assistant can access it, use the openai.Files.create() method. The response will include a file_id, which you need to attach to messages.

But first you need an openai API key. You can create a key at [platform.openai.com/api-keys](https://platform.openai.com/api-keys). I indend to use my API key in the future, therefore I will hide it using an environment variable. I will also hide any id that I will use.

In [97]:
import os

import openai
import dotenv
from openai import OpenAI

# Import environment variables
dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-4o-mini"

# Create session through openai client
client = OpenAI(api_key=api_key)

# 2. Managing Uploaded Files

You may want to manage the files you have uploaded to the model. To check what files have been uploaded use the **list** function. If you want to know the details of a file specifically **pass the file id the retrieve function**. If a file is no longer needed, the **delete** function removes the file with a specific file_id. Remember that multiple instances of the same file can be uploaded.

In [98]:
# What files are accessible to the Assistant
files = client.files.list()

# Delete all the files with a given name
file_name = "crypto.pdf"
for file in files:
    if file.filename == file_name:
        client.files.delete(file.id)

# Get all files with a given name
def get_files_with_filename(file_name):
    file_ids = []
    files = client.files.list()
    for file in files:
        if file.filename == file_name:
            file_ids.append(file.id)
    return file_ids

# Information about files with a given name
file_name = "public_perception_of_autonomous_vehicles.pdf"
file_ids = get_files_with_filename(file_name)
for id in file_ids:
    file_info = client.files.retrieve(id)
    print(file_info)

del files, file_ids

FileObject(id='file-9ryu7YKHZR3BcgYzvCd39M', bytes=275218, created_at=1741906032, filename='public_perception_of_autonomous_vehicles.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)


# 3. Vector Store

To access your files, the *file_search* tool uses the Vector Store object. Upload your files and create a Vector Store to contain them. Once the Vector Store is created, you should poll its status until all files are out of the *in_progress* state to ensure that all content has finished processing. The SDK provides helpers to uploading and polling in one shot.

In [96]:
# Create a vector store to group files
vector_store = client.beta.vector_stores.create(name="Knowledge Base")

# Ready the files for upload to OpenAI
file_paths = [
    "./knowledge_base/cripto.pdf",
    "./knowledge_base/public_perception_of_autonomous_vehicles.pdf"
]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector
# store, and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
# You can print the status and the file counts of the batch
# to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


You can also attach files as Message attachments on your thread. Doing so will create another vector_store associated with the thread, or, if there is already a vector store attached to this thread, attach the new files to the existing thread vector store. When you create a Run on this thread, the file search tool will query both the vector_store from your assistant and the vector_store on the thread.

In [102]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("./knowledge_base/cripto.pdf", "rb"), purpose="assistants"
)

# 3. Create an assistant

An assistant in OpenAI’s [Assistants API](https://platform.openai.com/docs/assistants/overview) is a specialized AI entity that can perform tasks, follow instructions, and use tools like code execution and file retrieval. Unlike a general LLM (Large Language Model), which is a standalone model responding to queries based purely on its training data, an assistant is a structured system that includes:


- A specific LLM (e.g., GPT-4-turbo) as its foundation.
- Custom instructions defining its behavior.
- Memory and threads for context retention over multiple interactions.
- Tools such as file handling and code execution to enhance functionality.

While an LLM is a raw model, an assistant is a wrapper around an LLM, fine-tuned with specific instructions and extended with capabilities such as retrieving uploaded files or running Python code.

**Note**: To enable files to be uploaded to an assistant, we first have to provide the *file_search* tool

In [93]:
# Create the assistant
assistant = client.beta.assistants.create(
    name="Demo Assistant",
    instructions="Process any file that is uploaded and provide insights",
    model="gpt-4o-mini",
    tools=[{"type": "code_interpreter"}, {"type": "file_search"}]
)
print("Assistant ID: ", assistant.id)

Assistant ID:  asst_2WT9TDtnTH749pRLPkteyLnk


To make the files accessible to your assistant, update the assistant’s *tool_resources* with the new vector_store id.

In [99]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# 4. Assistants API basics

Each conversation in the Assistants API happens inside a thread. Threads store the conversation history, allowing for context retention across multiple interactions.
To ensure the assistant can access an uploaded file, you need to attach it to a message in a thread.

If you want to restart a conversation just create a new thread. Although threads can be deleted explicitly, they are not stored for a long time, and as the last reference to the thread is lost so does the thread. OpenAI handles threads much like garbage manager does unused pointers

In [90]:
# Create a new thread for messages
thread = client.beta.threads.create()
thread_id = thread.id
print("The ID os the new thread is: ", thread_id)

The ID os the new thread is:  thread_Wbrt9hjeCNgDrPQdwzMUVevK


Now that you have a thread, send a message inside. Messages represent the user's input to the assistant within the thread. The assistant will respond based on the conversation history.

Since the API is asynchronous, you must start a run to make the assistant process the latest message. A run represents the assistant actively working on the request. Runs allow assistants to perform actions like retrieving files, running code, or generating text.

In [100]:

# Add a new message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="Summarize the files uploaded."
)

# Run the thread to generate a response to the last message
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant.id,
    instructions="The user has a deep knowledge about coding,"
                 "therefore your explanations should account for that",
    attachments=[
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
)
# Content sent to the API
print("The run has the following ID: ", run.id)

The run has the following ID:  run_jeLVsyu8bdjSx2062cA31uga


Once you send a message to the assistant, it processes the request asynchronously. This means you need to follow a structured approach to retrieve the response. The status of a response can be one of the following:

- **in_progress** → Assistant is still processing.
- **completed** → Assistant has finished, and a response is available.
- **failed** → The run failed due to an error.

Once the run is completed, retrieve the assistant’s response from the messages endpoint. The assistant’s response is stored in the thread, so you must fetch the latest messages to see what it generated.

In [101]:
import time

while True:
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread_id,
        run_id=run.id
    )
    print(f"Run Status: {run_status.status}")

    if run_status.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            print(f"{msg.role}: {msg.content[0].text.value}\n")
        break  # Stop polling once the assistant is done

    elif run_status.status == "failed":
        print("Run failed!")
        break
    time.sleep(2)  # Wait for 2 seconds before checking again

Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: completed
assistant: The uploaded file is titled "Public Perception of Autonomous Vehicles: A Brief Review." Here's a summary of its key points:

1. **Introduction to Autonomous Vehicles (AVs)**:
   - AVs are emerging as a transformative technology in the automotive industry, with the potential to enhance mobility, reduce emissions, and improve safety.
   - The success of AVs largely relies on public acceptance, which is influenced by various factors.

2. **Public Perception Factors**:
   - Several elements shape how different demographics perceive AVs, including age, education, gender, safety concerns, ethical considerations, and impacts from the COVID-19 pandemic.
   - Surveys indicate that younger, more educated males tend to have a more favorable view of AVs, whereas fears concerning safety and ethics can lead to negative attitudes.

3. **Demographic Insights**:
   - Age infl

# 5. Assistants File Search

File Search augments the Assistant with knowledge from outside its model, such as proprietary product information or documents provided by your users. OpenAI automatically parses and chunks your documents, creates and stores the embeddings, and use both vector and keyword search to retrieve relevant content to answer user queries.

[FileObject(id='file-F8niMSiNQCguJVAADySRTL', bytes=342004, created_at=1741897816, filename='cripto.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None), FileObject(id='file-KJV3eTZGm5jVBKjE16kGan', bytes=275218, created_at=1741888415, filename='public_perception_of_autonomous_vehicles.pdf', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)]
